# Evaluate model

## Imports

In [44]:
import pathlib
import re

import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image

## Config

In [40]:
MODEL_PATH = pathlib.Path(
    #"/media/cicheck/Extreme Pro/models/base/best_base_meso_net_05_0.14.h5"
    "/media/cicheck/Extreme Pro/models/old/base_new_new/best_base_meso_net_05_0.16.h5"
)
DATASET_PATH = pathlib.Path("/media/cicheck/Extreme Pro/datasets/modified/mod_1")

IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32

## Utils

In [53]:
prefix_pattern = re.compile(r"(?P<metric_prefix>\w+)((?P<metric_number>_\d+))?")


In [54]:
prefix_pattern.match("auc_1").group("metric_prefix")

'auc_1'

In [57]:
{
    1: 1
    if a == 1
    else 2: 2
    for a in (1, 2)
}

SyntaxError: invalid syntax (1670218073.py, line 4)

In [78]:
_metric_names_to_rephrased_names = {
    "binary_accuracy": "accuracy",
    "auc": "AUC",
    "precision": "precision",
    "recall": "recall",
    "true_positives": "TP",
    "true_negatives": "TN",
    "false_positives": "FP",
    "false_negatives": "FN",
}

def _rephrase_metrics_dict(metrics_dict: dict[str, float]) -> dict[str, float]:
    # Remove sufixes, e.g. auc_3 -> auc
    prefix_pattern = re.compile(r"(?P<metric_prefix>\w+)_(?P<metric_number>\d+)")
    # TODO: ugly comperhension
    normalized_metrics_dict = {
        (prefix_pattern.match(metric).group("metric_prefix") if prefix_pattern.match(metric) else metric): value
        for metric, value in metrics_dict.items()
    }
    return {
        (_metric_names_to_rephrased_names[metric] if metric in _metric_names_to_rephrased_names else metric): value
        for metric, value in metrics_dict.items()
    }

def _build_single_row(*columns, width):
    row_as_string = "|"
    for column in columns:
        row_as_string += column.ljust(width, " ")
        row_as_string += "|"
    row_as_string +="\n"
    return row_as_string

def print_metrics_dict_as_table(
    metrics_dict: dict[str, float], width: int = 30, precision: int = 5
):
    """Print metrics dictionay as markdown table.
    
    Args:
        metrics_dict: metrics dictionay (i.e. output of keras evaluate which output type set to dict).
        width: width of 
    """
    first_column_name = " metrics"
    second_column_name = " values"
    table_as_string = ""
    table_as_string += _build_single_row(first_column_name, second_column_name, width=width)
    header_sign = ":---:"
    table_as_string += _build_single_row(header_sign, header_sign, width=width)
    rephrased_metric_dict = _rephrase_metrics_dict(metrics_dict)
    for metric, value in rephrased_metric_dict.items():
        table_as_string += _build_single_row(metric, "{:.2f}".format(value), width=width)
    print(table_as_string)
        

In [79]:
print_metrics_dict_as_table(metrics_dict)

| metrics                      | values                       |
|:---:                         |:---:                         |
|loss                          |0.18                          |
|accuracy                      |0.95                          |
|auc_3                         |0.94                          |
|precision_3                   |0.73                          |
|recall_3                      |0.73                          |
|true_positives_3              |33441.00                      |
|true_negatives_3              |410275.00                     |
|false_positives_3             |12264.00                      |
|false_negatives_3             |12684.00                      |



## Setup model

In [35]:
evaluated_model = models.load_model(MODEL_PATH, compile=False)

In [36]:
METRICS = [
        metrics.BinaryAccuracy(),
        metrics.AUC(),
        metrics.Precision(),
        metrics.Recall(),
        metrics.TruePositives(),
        metrics.TrueNegatives(),
        metrics.FalsePositives(),
        metrics.FalseNegatives(),
    ]

In [37]:
# Model needs to be compiled again to set targeted metrices
optimizer = optimizers.Adam(
    learning_rate=1e-3,
    epsilon=1e-08
)

evaluated_model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=METRICS
)

## Evaluate whole dataset

In [41]:
evaluation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH.joinpath("test"),
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    label_mode="binary",
    class_names=["fakes", "reals"],
)

Found 468664 files belonging to 2 classes.


In [42]:
metrics_dict = evaluated_model.evaluate(evaluation_dataset, return_dict=True)

14646/14646 [==============================] - 496s 34ms/step - loss: 0.1848 - binary_accuracy: 0.9468 - auc_3: 0.9351 - precision_3: 0.7317 - recall_3: 0.7250 - true_positives_3: 33441.0000 - true_negatives_3: 410275.0000 - false_positives_3: 12264.0000 - false_negatives_3: 12684.0000


In [43]:
metrics_dict

{'loss': 0.18484479188919067,
 'binary_accuracy': 0.9467678070068359,
 'auc_3': 0.9351059198379517,
 'precision_3': 0.7316704988479614,
 'recall_3': 0.7250081300735474,
 'true_positives_3': 33441.0,
 'true_negatives_3': 410275.0,
 'false_positives_3': 12264.0,
 'false_negatives_3': 12684.0}

In [31]:
metrics_dict

{'loss': 0.15903404355049133,
 'binary_accuracy': 0.948237955570221,
 'auc_2': 0.9506139755249023,
 'precision_2': 0.7356554865837097,
 'recall_2': 0.7399458289146423,
 'true_positives_2': 34130.0,
 'true_negatives_2': 410275.0,
 'false_positives_2': 12264.0,
 'false_negatives_2': 11995.0}

In [ ]:
# Fajnie było widać że jedyne co się zmieniło to niektóre true positives przeszły na false negatives

## Evaluate single modification

In [ ]:
mod_name = 
mod_dir_path = DATASET_PATH / "test" / "reals" / mod_name
no_mod_images = len(list(mod_dir_path.iterdir()))
no_mod_images

In [ ]:
modifications_ds =  tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH.joinpath(f"test/reals/{mod_name}"),
    labels=(0 for _ in range(no_mod_images))
    label_mode="binary",
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
)

In [67]:
for x, y in evaluation_dataset:
    print(x[0].shape)
    print(x[0])
    print("XDDDDDD")
    print(y[0])
    break

(256, 256, 3)
tf.Tensor(
[[[194.99023  201.99023  215.99023 ]
  [194.98042  201.98042  215.96872 ]
  [193.96875  200.96875  212.97266 ]
  ...
  [223.97069  229.8828   248.86327 ]
  [229.9629   235.9629   254.9629  ]
  [224.01172  238.99414  255.      ]]

 [[189.97263  196.97263  210.97263 ]
  [187.93549  194.93549  208.93549 ]
  [182.94336  189.94336  203.94336 ]
  ...
  [223.96094  229.87305  248.85352 ]
  [228.9707   234.9707   253.9707  ]
  [224.00977  238.99219  254.99805 ]]

 [[185.89456  192.8848   206.9141  ]
  [181.88148  188.87172  202.90114 ]
  [175.01944  182.00967  196.0585  ]
  ...
  [223.92178  229.8339   248.81436 ]
  [228.92194  234.92194  253.92194 ]
  [223.96094  238.94336  254.94922 ]]

 ...

 [[ 45.9883    43.9883    39.9883  ]
  [ 45.        43.        39.      ]
  [ 45.        43.        39.      ]
  ...
  [ 55.00019   55.00019   55.00019 ]
  [ 54.015568  54.015568  54.015568]
  [ 54.009766  54.009766  54.009766]]

 [[ 45.003906  45.98633   38.009766]
  [ 44.01168

In [19]:
z

<tf.Tensor 'args_0:0' shape=(None, 256, 256, 3) dtype=float32>

In [ ]:
ehh = evaluation_dataset.filter(foo)

In [69]:
image_generator = image.ImageDataGenerator()

In [ ]:
image_generator.flow_from_directory()

In [ ]:
image_generator = next(img_gen.flow_from_directory(DATASET_PATH.joinpath("reals")))